# Investigation into "partially symbolic" execution for library identification
There are several papers now that explore the possibility of using symbolic execution to identify functions (and therefore, libraries) in stripped, statically linked executables. We want to explore this field a little more and see if there's room for innovation, as it may apply to VA. 

Note that much of this work will probably build off Nucleus, which is examined in more detail in `00_intro_lib_identify`.

## Introductory Thinking
We're drawing a lot of inspiration from two main works: [1] and [2]. While our method itself will probably end up most similar to [1], ~~we're interested in [2] ("Genius") because it demonstrated some pretty fantastical results.~~ We're now more interested in a quasi-follow-up work (shares 2 of the same authors as Genius), [3] a.k.a. _Gemeni_. 

[1] is interesting because it uses sampled I/O pairs to fingerprint basic blocks. Gemeni [3] uses a "graph embedding network" to generate some statistical features to train a neural net to recognize a function. My angle is to try to combine the approaches in a way that leverages the semantic-capturing abilities of I/O pairs and the general advantages of neural nets (or another ML-based solution). 

**Goals** (i.e. things that should be acheivable with this combined approach)
 - High accuracy (thanks to the NN)
 - Low query latency (i.e. a function should be classified in well under a second)
 - Capable of online incremental training (or really fast retraining)
 - Fingerprinting should happen relatively quickly (might require a custom partial execution engine)
 
## Basic Idea
Break an unknown function down into its basic blocks (or maybe don't? We could do this at the function level). Re-order each BB's instructions deterministically (could just sort instructions based on mnemonic) and "execute" the instructions in some predetermined environment (i.e. all registers initialized to some value). The resulting state of our environment becomes our "fingerprint" for that BB. Combine the FP's from the BBs in some way to get your function FP. Train a neural net or some other ML model with the function FP's.

How to do the actual "execution" is an open question. We could lift instructions to an IR like BIL or VEXT: this would give us the benefit of being cross-architecture, and we'd be able to turn BB's into equations that would be easy to plug concrete values into. But including a symbolic engine could possibly incur huge overhead, which would be a dealbreaker especially at the BB level (apps could have hundreds of thousands of BBs). I'm thinking we could build a custom execution engine that only implements instructions that would affect our environment meaningfully, but that could be costly in terms of development time (we'd have to implement a lot of the x86 ISA...). A good compromise might be to find a execution symbolic engine out in the wild and re-work it so that it's fast enough for us.

An "ideal" execution engine would work something like this: 
0. Recover a function's basic blocks, concatenate them all together, strip out all control flow instructions, and sort by instruction ID
1. Convert arbitrary blocks of instructions into some IR
   * We assume this IR does away with instructions that don't affect register or memory values (like nops, IRQs, all control flow instructions, etc).
   * Exactly what to do with function calls is also an open question. There exists several functions that do little other than a `call`, so removing them blindly would cause those functions to be unrecognizable. That might not be the worst thing in the world, as there's probably little reason to know about those wrapper functions anyways. 
2. Iterate through instructions, map registers to use least possible index (e.g. say a block of instructions contains references to R4, R18, and R31. All references would be remapped to R0, R1, R2, respectively), and intialize "virtual" registers with some preset values (maybe just register number = value?).
3. Execute the instructions sequentially. Any reads from memory addresses that haven't been written to yet return the address as the value.
   * The resulting register & memory context (i.e. the result of the function's calculations) alongside the number of registers and memory values written to becomes the final function fingerprint.
   
A baseline execution engine (for initial testing) could be much simpler by just using a stock X86 context and not remapping register numbers. Also, the engine could be written in Python initially, but should probably be rewritten in a compiled language.

It might also be useful to repeat the process several times with different inputs (researching this....)

## Caveat
Our intended usage model looks something like this:
* User uploads a stripped, statically linked binary to VA
* VA uses Nucleus to provide our method with function boundaries
* Our method analyzes the functions and (having already been trained) identifies them with identifiers like `connect() from openssl v2.2.1-2.4.2` (notice the range of versions
* VA cross references the output our method with _some database_ of vulnerable functions

The key here is the last bullet: we've yet to find a database of vulnerable functions. CVE's rarely list the vulnerable function (just the application version), and even when they do, it's not in a standardized format (so not easily machine-readable). Without such a database, we're left simple with function names, library names, and library version ranges (which could be quite large). Say, for example, that a CVE tells us that version 2.3.2 of `openssl` was found to be insecure. In our example above, our `connect()` function could possibly be from that version (since it's in the range of 2.2-2.4), so does that mean that this binary is vulnerable? I think an answer of 'yes' would very quickly lead to "taint explosion," i.e. everything except the very latest version of a library will be considered vulnerable.

This probably isn't a showstopping problem: function names and version ranges are still very useful pieces of information. And a second-level filter (like searching for a version string) could provide the granularity needed to narrow down to a single library version, which is enough for VA. One could also reason about the ranges: if we know that `connect()` does not change between v2.2 and v2.4, and the CVE-in-question mentions 2.3.2 as the only vulnerable version, then we know that `connect()` cannot be vulnerable (otherwise v2.2-2.3.2 would have been mentioned by the CVE, and versions after 2.3.2 would have patched the function). In other words, the set of versions in which a particular implementation of a function exists must be a subset of the versions mentioned by the CVE in order for the function to be considered vulnerable.

## Works Cited
[1] Pewney et al. [Cross-Architecture Bug Search in Binary Executables](https://ieeexplore.ieee.org/document/7163056/) (2015)

[2] Feng et al. [Scalable Graph-based Bug Search for Firmware Images](https://dl.acm.org/citation.cfm?id=2978370) _(a.k.a. Genius)_ (2016)

[3] Xu et al. [Neural Network-based Graph Embedding for Cross-Platform Binary Code Similarity Detection](https://arxiv.org/abs/1708.06525) _(a.k.a. Gemeni)_ (2017)

[4] Andriesse, Sowinska, Bos. Compiler Agnostic Function Detection in Binaries _(a.k.a. Nucleus)_ (2017)

## Other Related Work
In addition to the papers linked above, the following works may be of interest

- Alrabaee et al. [FOSSIL: A Resilient and Efficient System for Identifying FOSS Functions in Malware Binaries](https://dl.acm.org/citation.cfm?id=3175492) (2018)
  - Statistical-feature-heavy function ID model designed to be resilient to "light obfuscation". Pretty decent accuracy and mediocre speed (15-60 sec/binary).
  - Useful table (#14) on page 8:30 showing comparison of most recent function ID solutions
